In [1]:
from scipy.io import loadmat
import numpy as np
import mne
from src import preprocess,dtw_similarity,qt_clustering,visualize
%matplotlib qt

#  Load DEAP dataset

In [2]:
def load_deap_mat(file_path):
    """
    Load a preprocessed DEAP .mat file.

    Args:
        file_path (str): Path to the .mat file.

    Returns:
        dict: Dictionary containing EEG data and labels.
    """
    # Load the .mat file
    mat_data = loadmat(file_path)
    
    # Example structure: adjust keys based on your .mat file's content
    eeg_data = mat_data['data']  # EEG signals (e.g., channels x time)
    labels = mat_data['labels']  # Corresponding labels or metadata

    return {'eeg_data': eeg_data, 'labels': labels}

In [3]:
deap_channels = [
    "Fp1", "AF3", "F3", "F7", "FC5", "FC1", "C3", "T7", "CP5", "CP1",
    "P3", "P7", "PO3", "O1", "Oz", "Pz", "Fp2", "AF4", "Fz", "F4",
    "F8", "FC6", "FC2", "Cz", "C4", "T8", "CP6", "CP2", "P4", "P8",
    "PO4", "O2"
]

file_path = "../../../Database/DEAP/data_preprocessed_matlab/s01.mat"
data = load_deap_mat(file_path)

eeg_data = preprocess.preprocess_eeg_data(data['eeg_data']) #去除前3秒的数据
# labels = data['labels']
eeg_data = eeg_data[:32,:int(128*2)] #只选取第一次实验前十秒的数据

# print(f"EEG data shape: {eeg_data.shape}")
# print(f"Labels shape: {labels.shape}")


EEG data shape: (32, 256)


In [ ]:
# print(f"eeg_data[0] shape: {eeg_data[0].shape}")
eeg_dtw_matrix = dtw_similarity.compute_dtw_similarity(eeg_data, 'minmax')
labels = qt_clustering.qt_clustering(eeg_dtw_matrix,0.75)
# print(f"labels: {labels}")

In [5]:
# Creating channel layouts for standard 10-20 systems
montage = mne.channels.make_standard_montage('easycap-M1')

# Get Channel Name
channel_names = montage.ch_names
print("Available channels:", channel_names)

# Extract the 3D coordinates of the channel
subset_positions = {ch: montage.get_positions()['ch_pos'][ch] for ch in deap_channels if ch in montage.ch_names}

# Create a new subset of Montage
subset_montage = mne.channels.make_dig_montage(
    ch_pos=subset_positions,
    coord_frame='head'
)

# Plotting the original subset of channel layouts
# subset_montage.plot()

# Setting the zoom ratio (down to 90%)
scaling_factor = 0.9

# Zoom Channel Position
scaled_positions = {ch: pos * scaling_factor for ch, pos in subset_positions.items()}

# Creating a scaled subset of Montage
scaled_subset_montage = mne.channels.make_dig_montage(
    ch_pos=scaled_positions,
    coord_frame='head'
)

# Plotting the scaled subset channel layout
# scaled_subset_montage.plot()

Available channels: ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'F1', 'F2', 'FC1', 'FC2', 'C1', 'C2', 'CP1', 'CP2', 'P1', 'P2', 'AF3', 'AF4', 'FC3', 'FC4', 'CP3', 'CP4', 'PO3', 'PO4', 'F5', 'F6', 'FC5', 'FC6', 'C5', 'C6', 'CP5', 'CP6', 'P5', 'P6', 'AF7', 'AF8', 'FT7', 'FT8', 'TP7', 'TP8', 'PO7', 'PO8', 'F9', 'F10', 'FT9', 'FT10', 'TP9', 'TP10', 'P9', 'P10', 'PO9', 'PO10', 'O9', 'O10', 'Fpz', 'AFz', 'FCz', 'CPz', 'POz', 'Oz', 'Iz']


# Visualization

In [6]:
# Draw hot map of similarity matrix
visualize.plot_similarity_matrix(eeg_dtw_matrix,deap_channels)

# Draw the results of classification
visualize.plot_clusters(labels,scaled_positions)